In [3]:
import os
import jax
import jax.numpy as jnp
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [2]:
pd.read_csv('/n/holystore01/LABS/iaifi_lab/Lab/quijote_bsq/halos_1234.csv')

,x,y,z,vx,vy,vz,Jx,Jy,Jz,M200c,Rvir
0,523.87921,997.09070,329.35129,-58.62,429.58,121.10,-7.548000e+17,5.696000e+17,-9.084000e+17,4051300000000000,3264.635
1,832.85913,388.96509,288.60141,575.17,1124.71,638.53,-9.456000e+17,-2.231000e+17,4.367000e+17,3363200000000000,3161.956
2,931.41461,481.97626,815.59155,6.48,139.10,20.07,-3.455000e+17,-3.971000e+15,3.417000e+17,3023700000000000,2941.829
3,830.04327,74.93627,536.11200,653.95,41.30,-186.97,-1.212000e+17,1.092000e+18,-8.631000e+16,2881700000000000,2889.544
4,245.69009,977.35895,225.14355,781.71,162.50,181.48,-2.718000e+15,7.374000e+17,6.526000e+17,2784700000000000,2836.105
...,...,...,...,...,...,...,...,...,...,...,...
4995,259.89481,331.97177,943.88849,589.36,-189.89,-11.83,4.858000e+15,2.160000e+16,-1.090000e+15,342250000000000,1462.600
4996,851.34479,954.74152,351.61807,-123.16,-530.66,171.77,-3.083000e+16,1.434000e+16,-7.996000e+15,342250000000000,1421.951
4997,398.50443,974.74878,164.84859,572.37,578.25,611.00,4.723000e+15,-1.924000e+16,3.721000e+16,342250000000000,1458.534
4998,841.52911,938.93311,396.58209,952.90,-270.64,-439.37,8.968000e+15,7.282000e+15,6.092000e+15,342250000000000,1461.585


In [28]:
np.loadtxt('/n/holystore01/LABS/iaifi_lab/Lab/quijote_bsq/BSQ_params.txt')

array([[0.28225995, 0.02676735, 0.71647832, 0.84342318, 0.97336542],
       [0.36990511, 0.07379601, 0.68728997, 1.1230911 , 0.79291034],
       [0.40535655, 0.04417873, 0.89734204, 0.99043181, 0.68721196],
       ...,
       [0.4053684 , 0.05990108, 0.78936934, 1.07437532, 0.85520303],
       [0.36991773, 0.02950744, 0.57599691, 0.83866428, 0.97491965],
       [0.28227181, 0.06916839, 0.8218082 , 1.12140837, 0.79131186]])

In [21]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def serialize_example(features):
    """
    Creates a tf.train.Example message ready to be written to a file.
    """
    
    feature = {k: _bytes_feature(tf.io.serialize_tensor(tf.cast(v, tf.float32)).numpy()) for k, v in features.items()}
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def process_csv_to_tfrecords(folder_path, num_tfrecords):
    """
    Converts CSV files in a folder to a specified number of TFRecord files.
    """
    csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    num_files = len(csv_files)
    files_per_tfrecord = int(np.ceil(num_files / num_tfrecords))
    
    for i in range(2):
        tfrecord_file = os.path.join('/n/holystore01/LABS/iaifi_lab/Lab/quijote_bsq_tfrecords', f'points_{i + 1}.tfrecord')
        with tf.io.TFRecordWriter(tfrecord_file) as writer:
            subset_csv_files = csv_files[i * files_per_tfrecord:(i + 1) * files_per_tfrecord]
            for csv_file in tqdm(subset_csv_files, desc=f'Writing TFRecord {i + 1} / {num_tfrecords}'):
                df = pd.read_csv(csv_file)
                features = {col: df[col] for col in df.columns if col != 'id'}
                example = serialize_example(features)
                writer.write(example)

folder_path = '/n/holystore01/LABS/iaifi_lab/Lab/quijote_bsq'
num_tfrecords = 10  # Specify the number of TFRecord files you want

process_csv_to_tfrecords(folder_path, num_tfrecords)

Writing TFRecord 2 / 10: 100%|██████████| 1818/1818 [00:21<00:00, 83.23it/s]


In [4]:
def _parse_function(proto, features=['x', 'y', 'z', 'vx', 'vy', 'vz', 'M200c']):
    # Define your tfrecord again. It must match with how you wrote it
    keys_to_features = {k: tf.io.FixedLenFeature([], tf.string) for k in features}

    # Load one example
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)

    # Convert each feature from a serialized string to a tensor and store in a list
    feature_tensors = [tf.io.parse_tensor(parsed_features[k], out_type=tf.float32) for k in features]

    # Stack the feature tensors to create a single tensor
    # Each tensor must have the same shape
    stacked_features = tf.stack(feature_tensors, axis=1)  # Creates a [num_points, num_features] tensor

    return stacked_features

In [24]:
from functools import partial

batch_size = 64

files = tf.io.gfile.glob(f"/n/holystore01/LABS/iaifi_lab/Lab/quijote_bsq_tfrecords/*.tfrecord")
dataset = tf.data.TFRecordDataset(files)

num_total = sum(1 for _ in dataset)

features = ['x', 'y', 'z', 'vx', 'vy', 'vz', 'Jx', 'Jy', 'Jz',  'M200c', 'Rvir']

dataset = dataset.map(partial(_parse_function, features=features))
dataset = dataset.batch(batch_size)

In [27]:
iterator = iter(dataset)

# First pass: Compute the mean
mean = 0
for i in tqdm(range(num_total // batch_size)):
    batch = next(iterator)
    batch = tf.cast(batch, tf.float64)
    mean += tf.reduce_mean(batch, axis=(0, 1))
mean /= (num_total // batch_size)

# Reset the iterator for the second pass
iterator = iter(dataset)

# Second pass: Compute the variance, then standard deviation
variance = 0
for i in tqdm(range(num_total // batch_size)):
    batch = next(iterator)
    batch = tf.cast(batch, tf.float64)
    variance += tf.reduce_mean(tf.square(batch - mean), axis=(0, 1))
variance /= (num_total // batch_size)
std = tf.sqrt(variance)

print(mean, std)

  0%|          | 0/56 [00:00<?, ?it/s]

100%|██████████| 56/56 [00:00<00:00, 72.15it/s]


tf.Tensor(
[ 5.00015036e+02  4.99979081e+02  4.99834963e+02 -4.01212363e-02
 -2.45797723e-02 -2.11057118e-02 -2.08748973e+13  6.18134186e+12
  3.01321159e+13  3.23751137e+14  1.42657628e+03], shape=(11,), dtype=float64) tf.Tensor(
[2.88780224e+02 2.88736648e+02 2.88677205e+02 3.42325047e+02
 3.42505664e+02 3.42552337e+02 5.85229988e+16 5.77894413e+16
 5.73338974e+16 2.59518748e+14 2.96896618e+02], shape=(11,), dtype=float64)
